<a href="https://colab.research.google.com/github/yuyuyu-26/DQ-practice.rb/blob/master/cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

概要・考察<br>
今回はCIFAR-10と呼ばれる画像データ(5万枚の32x32ピクセルのカラーの画像データと、それを分類する10個のラベル)を使用して、画像分類問題に取り組みました。結果から言うと、学習結果は81.54%の精度で分類、学習させたモデルを使用してのテストデータの分類は80.72%での精度の分類をすることができました。初めてのモデル作成にしては自分なりには頑張ったと思います。<br>
実装に当たっては分類問題であったため、softmax関数を使用してone-hotベクトルにてわかりやすいようにしたところ/学習させる際のエポック数は10にしようとしていましたが、少なすぎると上手く学習されないと考え40にしました(実行に2時間)。<br>
今回出た精度をさらに高めていくためには、過学習についてもより考えなくてはならないと思いました。そのためには損失関数の動きをよく見て、KerasではEarly stoppingというメソッドがあるみたいなので、これを上手く使用すればまたさらに良くなりそうかな？と思います。<br>
今回の初実装は講義で学んだ基礎知識を使用しつつ、Kerasのドキュメントを読みながら行いました。基本的にKerasのドキュメントにどうコードを書けば実装できるのか書いてあったので、それを参考にしつつ実装した結果、なんとか作成できました。


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import dlt
import os

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

In [5]:
data = dlt.cifar.load_cifar10()

170500096/170498071 [==============================] - 3s 0us/step


In [3]:
pip install dlt

In [6]:
#学習用の画像とラベル
print(data.train_images.shape)
print(data.train_labels.shape)

#テスト用の画像とラベル
print(data.test_images.shape)
print(data.test_labels.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [7]:
dlt.utils.plot_examples(data, fname='examples.png')

In [8]:
X_train = data.train_images.reshape([-1, 32, 32, 3])
X_test = data.test_images.reshape([-1, 32, 32, 3])

print('%i training samples' % X_train.shape[0])
print('%i test samples' % X_test.shape[0])

print(X_train.shape)
print(X_test.shape)

50000 training samples
10000 test samples
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [9]:
X_train = X_train.astype('float32') / 255 #0～1までの間に正規化
X_test = X_test.astype('float32') / 255

In [10]:
print(data.train_labels[:5])

[[6]
 [9]
 [9]
 [4]
 [1]]


In [11]:
Y_train = to_categorical(data.train_labels, 10)
Y_test = to_categorical(data.test_labels, 10)

In [12]:
num_classes = 10
model = Sequential()
# CNN layer 1
model.add(Conv2D(32, (3, 3), padding='same', #conv2DにてCNN実装
                 input_shape=X_train.shape[1:])) #出力画像のサイズが変わらないように「padding=’same’」でパディングを実施。
model.add(Activation('relu')) #relu関数使用
model.add(Conv2D(32, (3, 3))) #3×3のフィルターを32枚
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #「2×2」の大きさの最大プーリング層。入力画像内の「2×2」の領域で最大の数値を出力する。
model.add(Dropout(0.25)) #ドロップアウトによる過学習予防。全結合の層とのつながりを「25%」無効化。

# CNN layer 2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# output
model.add(Flatten()) #1次元ベクトルに変換
model.add(Dense(512)) #全結合層。出力は512。
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes)) #num_classes=10。0～9までの数値で判定するため。
model.add(Activation('softmax')) #softmax関数により、総和が1になるよう調整

In [13]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        1

・学習<br>
各コードに詳細記載

In [14]:
model.compile( #compile()でプロセス設定
loss='categorical_crossentropy', # 損失関数の設定
optimizer=Adam(lr=0.001), # 最適化法の指定
metrics=['accuracy']) #評価関数の指定

In [15]:
fit = model.fit(X_train, Y_train, #fit()で訓練実行
              batch_size=128,
              epochs=40, #10だと少ない？
              verbose=1,
              validation_split=0.1 # 今回は訓練データセットの10%をvalidationデータセットとして使う
                )

Epoch 1/40
352/352 [==============================] - 176s 498ms/step - loss: 1.8722 - accuracy: 0.3076 - val_loss: 1.2784 - val_accuracy: 0.5448
Epoch 2/40
352/352 [==============================] - 175s 497ms/step - loss: 1.2806 - accuracy: 0.5428 - val_loss: 1.1211 - val_accuracy: 0.6040
Epoch 3/40
352/352 [==============================] - 175s 498ms/step - loss: 1.0920 - accuracy: 0.6107 - val_loss: 0.9129 - val_accuracy: 0.6846
Epoch 4/40
352/352 [==============================] - 175s 497ms/step - loss: 0.9626 - accuracy: 0.6613 - val_loss: 0.8387 - val_accuracy: 0.7114
Epoch 5/40
352/352 [==============================] - 175s 497ms/step - loss: 0.8776 - accuracy: 0.6874 - val_loss: 0.7660 - val_accuracy: 0.7330
Epoch 6/40
352/352 [==============================] - 176s 499ms/step - loss: 0.8130 - accuracy: 0.7128 - val_loss: 0.7451 - val_accuracy: 0.7440
Epoch 7/40
352/352 [==============================] - 175s 498ms/step - loss: 0.7632 - accuracy: 0.7311 - val_loss: 0.6932 -

学習させた結果としておよそ81.54%の精度で分類できることがわかりました。

・評価<br>
詳細は各コードに記載

In [16]:
score = model.evaluate(X_test, Y_test, #evaluate()にて学習されたモデルがどの程度の精度を持ってテストデータを判定できるか(分類できるか)評価。
                    verbose=0
                    )

print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.661607563495636
Test accuracy: 0.8072999715805054


学習させた結果を用いてテストデータを分類させると80.72%の精度で正しく分類されることがわかりました。

In [17]:
folder = 'results'
if not os.path.exists(folder):
    os.makedirs(folder)

model.save(os.path.join(folder, 'my_model.h5'))

In [19]:
preds = model.predict(X_test)
cls = np.argmax(preds,axis=1)
for i in range(10):
    dlt.utils.plot_prediction(
        preds[i],
        data.test_images[i],
        data.test_labels[i],
        data.classes,
        fname=os.path.join(folder, 'test-%i.png' % i))

In [20]:
dlt.utils.plot_confusion_matrix(data.test_labels, cls, data.classes,
                                  title='confusion matrix',
                                  fname=os.path.join(folder, 'confusion_matrix.png'))